In [14]:
from pyspark.sql import SparkSession
from pyspark import SparkConf


In [15]:
spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.jars", "https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.13/mysql-connector-java-8.0.13.jar, https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector-assembly_2.12/3.4.0/spark-cassandra-connector-assembly_2.12-3.4.0.jar, https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector_2.12/3.4.0/spark-cassandra-connector_2.12-3.4.0.jar") \
    .config("spark.cassandra.connection.host", "cassandra.default.svc.cluster.local") \
    .config("spark.cassandra.connection.port", "9042") \
    .getOrCreate()

In [16]:
df = spark.read \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .load()
df.show()

+-------+-------+-------+----------+--------------+---------------+-------------+--------------+-------------+---------+------------+-------------+------------------+-------------+--------------+------------+--------------------+--------------------+--------------+--------------+-------------------+--------------------+----------------+---------+-----------------------------------+--------+-------------------+---------------+---------------------+-----------+-----------+----------+-------------+--------+-----------+----------+----------+---------------+----------------+----------------+---------+------------+---------+--------+-------------+------------+----------------+----------------+---------------+------------+--------------------+------------+------------+-----------+-------------------------------------+----------+----------+-----------+-----------+---------------+----------+----------+------------+------------+------------+---------------+------------+-----------+-----------+--

In [17]:
df.count()

145975

In [18]:
df.printSchema()

root
 |-- mem_no: integer (nullable = true)
 |-- mem_sex: string (nullable = true)
 |-- mem_loc: string (nullable = true)
 |-- mem_l_code: string (nullable = true)
 |-- mem_birth_year: integer (nullable = true)
 |-- mem_birth_month: integer (nullable = true)
 |-- mem_birth_day: integer (nullable = true)
 |-- mem_birth_date: date (nullable = true)
 |-- mem_birth_ddi: integer (nullable = true)
 |-- mate_slct: string (nullable = true)
 |-- mate_slct_yn: string (nullable = true)
 |-- mem_name_slct: string (nullable = true)
 |-- mem_last_name_code: integer (nullable = true)
 |-- mem_last_name: string (nullable = true)
 |-- mem_first_name: string (nullable = true)
 |-- rprsn_mov_yn: string (nullable = true)
 |-- rprsn_mov: string (nullable = true)
 |-- rprsn_mov_img: string (nullable = true)
 |-- rprsn_photo_no: integer (nullable = true)
 |-- rprsn_photo_yn: string (nullable = true)
 |-- rprsn_photo_date: timestamp (nullable = true)
 |-- rprsn_photo: string (nullable = true)
 |-- rprsn_chrgr

In [19]:
# t = t.replace('(nullable = true)', '')
# t = t.replace('|--' ,'')
# t = t.replace('\n', '')
# t = t.strip()
# t = t.replace('   ', ',')
# t = t.replace(': ', ':')
# t = t.replace(':integer', ' int')
# t = t.replace(':string', ' text')
# t = t.replace(':timestamp', ' timestamp')
# t = t.replace(':date', ' date')
# t

In [24]:
selected_columns = ["mem_no","mem_sex","mate_conts","family_conts","conts_upd_cnt","family_conts_upd_cnt","upd_date","ins_date"]
selected_df = df.select(*selected_columns)
selected_df.show()

+-------+-------+-----------------------------------+-------------------------------------+-------------+--------------------+-------------------+-------------------+
| mem_no|mem_sex|                         mate_conts|                         family_conts|conts_upd_cnt|family_conts_upd_cnt|           upd_date|           ins_date|
+-------+-------+-----------------------------------+-------------------------------------+-------------+--------------------+-------------------+-------------------+
|      5|      m|삼남매이며 형제간에우애하고 화목...|  삼남매이며 형제간에우애하고 화목...|            6|                  11|2021-03-11 11:35:22|2015-07-22 16:31:35|
| 958237|      m|  음.마음.통하고.서로배려하고.진...|    현재는.혼자임.가족은.부모님.두...|            0|                   0|2020-07-17 14:30:42|2019-10-29 13:19:38|
|1241232|      f|   나는 낙천적입니다\r\n매사예많...|    나는 건강합니다\n나는 낙천적입...|            1|                   0|2020-07-17 14:30:42|2020-07-07 12:01:05|
|      2|      m|   화려한 돌씽입니다. 맘 맞는분 ...|    부모님 두분 아직은 건강하시고 ...|            1

In [ ]:
selected_df.write \
    .format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table="usertable", keyspace="pn") \
    .save()

# 작업 완료 후 SparkSession 종료
spark.stop()

In [ ]:
# df.write \
#     .format("org.apache.spark.sql.cassandra") \
#     .mode("overwrite") \
#     .option("confirm.truncate","true") \
#     .options(table="user", keyspace="test") \
#     .save()

# # 작업 완료 후 SparkSession 종료
# spark.stop()